# Install Event Hub packages

In [ ]:
! python --version

! pip install azure-eventhub==5.11.5 azure-identity --upgrade --force --quiet

StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 9, Finished, Available, Finished)

Python 3.11.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nni 3.0 requires filelock<3.12, but you have filelock 3.13.1 which is incompatible.
datasets 2.19.1 requires fsspec[http]<=2024.3.1,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.


# Connect to Event Stream

## Azure Managed Identity Setup Requirements

**For this to work, you need to:**

1. **Enable Managed Identity** on your Azure service:
   - Azure App Service: Go to Identity → System assigned → On
   - Azure Function App: Go to Identity → System assigned → On  
   - Azure Kubernetes Service (AKS): Use Azure AD Pod Identity or Workload Identity
   - Azure VM: Enable system-assigned managed identity

2. **Grant permissions** to your Event Hub:
   - Go to your Event Hub Namespace → Access control (IAM)
   - Add role assignment: "Azure Event Hubs Data Sender"
   - Assign to your managed identity

3. **For local development:**
   - Install Azure CLI: `az login` 
   - Or use Visual Studio/VS Code Azure authentication
   - `DefaultAzureCredential()` will automatically use your local credentials

**No connection strings or secrets needed!** 🔒

In [ ]:
import json
import os
import socket
import random
from random import randrange
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import DefaultAzureCredential

# Azure Managed Identity Configuration (Secure - No hardcoded secrets)
eventhub_namespace = "esehusw32tv7aa7zx6wldmh5.servicebus.windows.net"
eventhub_name = "es_7124688f-ba72-4974-a1b6-6baaf53995fb"

# Use Azure Managed Identity for authentication
credential = DefaultAzureCredential()
producer_events = EventHubProducerClient(
    fully_qualified_namespace=eventhub_namespace,
    eventhub_name=eventhub_name,
    credential=credential
)

hostname = socket.gethostname()
print(f"Connected to Event Hub using Azure Managed Identity from {hostname}")

StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 10, Finished, Available, Finished)

In [ ]:
# Test the Managed Identity connection
try:
    # Try to create a test batch to verify connection
    test_batch = producer_events.create_batch()
    print("✅ Azure Managed Identity authentication successful!")
    print(f"✅ Connected to Event Hub: {eventhub_name}")
    print(f"✅ Namespace: {eventhub_namespace}")
except Exception as e:
    print("❌ Managed Identity authentication failed!")
    print(f"Error: {e}")
    print("\n🔧 Troubleshooting steps:")
    print("1. Ensure Managed Identity is enabled on your Azure service")
    print("2. Grant 'Azure Event Hubs Data Sender' role to the managed identity")
    print("3. For local development, run 'az login' first")
    print("4. Check that the Event Hub namespace and name are correct")

# Send events to Event Stream

In [ ]:
def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    #print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 11, Finished, Available, Finished)

# Wrapper to generate events and send to event Hub

In [ ]:
us_states = {
    "Alabama": {"capital": "Montgomery", "latitude": 32.361538, "longitude": -86.279118},
    "Alaska": {"capital": "Juneau", "latitude": 58.301935, "longitude": -134.419740},
    "Arizona": {"capital": "Phoenix", "latitude": 33.448457, "longitude": -112.073844},
    "Arkansas": {"capital": "Little Rock", "latitude": 34.736009, "longitude": -92.331122},
    "California": {"capital": "Sacramento", "latitude": 38.555605, "longitude": -121.468926},
    "Colorado": {"capital": "Denver", "latitude": 39.739167, "longitude": -104.984167},
    "Connecticut": {"capital": "Hartford", "latitude": 41.767, "longitude": -72.677},
    "Delaware": {"capital": "Dover", "latitude": 39.161921, "longitude": -75.526755},
    "Florida": {"capital": "Tallahassee", "latitude": 30.4518, "longitude": -84.27277},
    "Georgia": {"capital": "Atlanta", "latitude": 33.76, "longitude": -84.39},
    "Hawaii": {"capital": "Honolulu", "latitude": 21.30895, "longitude": -157.826182},
    "Idaho": {"capital": "Boise", "latitude": 43.613739, "longitude": -116.237651},
    "Illinois": {"capital": "Springfield", "latitude": 39.78325, "longitude": -89.650373},
    "Indiana": {"capital": "Indianapolis", "latitude": 39.790942, "longitude": -86.147685},
    "Iowa": {"capital": "Des Moines", "latitude": 41.590939, "longitude": -93.620866},
    "Kansas": {"capital": "Topeka", "latitude": 39.04, "longitude": -95.69},
    "Kentucky": {"capital": "Frankfort", "latitude": 38.197274, "longitude": -84.86311},
    "Louisiana": {"capital": "Baton Rouge", "latitude": 30.45809, "longitude": -91.140229},
    "Maine": {"capital": "Augusta", "latitude": 44.323535, "longitude": -69.765261},
    "Maryland": {"capital": "Annapolis", "latitude": 38.972945, "longitude": -76.501157},
    "Massachusetts": {"capital": "Boston", "latitude": 42.2352, "longitude": -71.0275},
    "Michigan": {"capital": "Lansing", "latitude": 42.7335, "longitude": -84.5467},
    "Minnesota": {"capital": "Saint Paul", "latitude": 44.95, "longitude": -93.094},
    "Mississippi": {"capital": "Jackson", "latitude": 32.32, "longitude": -90.207},
    "Missouri": {"capital": "Jefferson City", "latitude": 38.572954, "longitude": -92.189283},
    "Montana": {"capital": "Helena", "latitude": 46.595805, "longitude": -112.027031},
    "Nebraska": {"capital": "Lincoln", "latitude": 40.809868, "longitude": -96.675345},
    "Nevada": {"capital": "Carson City", "latitude": 39.160949, "longitude": -119.753877},
    "New Hampshire": {"capital": "Concord", "latitude": 43.220093, "longitude": -71.549127},
    "New Jersey": {"capital": "Trenton", "latitude": 40.221741, "longitude": -74.756138},
    "New Mexico": {"capital": "Santa Fe", "latitude": 35.667231, "longitude": -105.964575},
    "New York": {"capital": "Albany", "latitude": 42.659829, "longitude": -73.781339},
    "North Carolina": {"capital": "Raleigh", "latitude": 35.771, "longitude": -78.638},
    "North Dakota": {"capital": "Bismarck", "latitude": 46.8133, "longitude": -100.779},
    "Ohio": {"capital": "Columbus", "latitude": 39.962245, "longitude": -83.000647},
    "Oklahoma": {"capital": "Oklahoma City", "latitude": 35.482309, "longitude": -97.534994},
    "Oregon": {"capital": "Salem", "latitude": 44.931109, "longitude": -123.029159},
    "Pennsylvania": {"capital": "Harrisburg", "latitude": 40.269789, "longitude": -76.875613},
    "Rhode Island": {"capital": "Providence", "latitude": 41.82355, "longitude": -71.422132},
    "South Carolina": {"capital": "Columbia", "latitude": 34.000, "longitude": -81.035},
    "South Dakota": {"capital": "Pierre", "latitude": 44.367966, "longitude": -100.336378},
    "Tennessee": {"capital": "Nashville", "latitude": 36.165, "longitude": -86.784},
    "Texas": {"capital": "Austin", "latitude": 30.266667, "longitude": -97.75},
    "Utah": {"capital": "Salt Lake City", "latitude": 40.7547, "longitude": -111.892622},
    "Vermont": {"capital": "Montpelier", "latitude": 44.260059, "longitude": -72.575386},
    "Virginia": {"capital": "Richmond", "latitude": 37.54, "longitude": -77.46},
    "Washington": {"capital": "Olympia", "latitude": 47.042418, "longitude": -122.893077},
    "West Virginia": {"capital": "Charleston", "latitude": 38.349497, "longitude": -81.633294},
    "Wisconsin": {"capital": "Madison", "latitude": 43.074722, "longitude": -89.384444},
    "Wyoming": {"capital": "Cheyenne", "latitude": 41.145548, "longitude": -104.802042}
}



StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 12, Finished, Available, Finished)

In [ ]:
import random, math, json
import random
import time
import uuid
from datetime import datetime, timedelta

class FraudDetectionSimulator:
    def __init__(self, current_time_pas):
        # self.grid_id = grid_id
        self.unique_id = str(uuid.uuid4())
        self.current_time = current_time_pas

    def log_event(self, activity, sensor_data):
        event = {
            "timestamp": self.current_time.strftime("%Y-%m-%d %H:%M:%S"),
            "event_id": self.unique_id,
            "source": activity,
            # "grid_id": grid_id,
            "sensor_data": sensor_data
        }
        log_event = event
        log_obj = json.dumps(log_event)
        #impressionEvent = json_object    
        sendToEventsHub(log_event, producer_events)
        #print(log_event)

    def get_sensor_data(self, source, state, details):
        #sources = ["Wire Transfers", "ACH Files", "Check", "ATM Transactions", "Real-Time Payments"]
        data = {}

        # for source in sources:
        Transactions_value = round(random.uniform(300, 500) if random.random() < 1/1000 else random.uniform(100, 300), 0)

        # Ensure Fraud < Transaction only 1 in 10 times
        if random.random() > 1/20:
            Fraud_value = round(random.uniform(0.5 * Transactions_value, Transactions_value), 0)
        else:
            Fraud_value = round(random.uniform(1.1 * Transactions_value, 2 * Transactions_value), 0)

        # Calculate Suspicious Transactions based on the difference
        percentage = round((Fraud_value / Transactions_value) * 100, 0)

        if percentage < 70:
            Suspicious_Transactions = round(random.uniform(-10, 30), 0)
        elif percentage < 80:
            Suspicious_Transactions = round(random.uniform(30, 50), 0)
        elif percentage < 90:
            Suspicious_Transactions = round(random.uniform(50, 60), 0)
        elif percentage < 100:
            Suspicious_Transactions = round(random.uniform(60, 100), 0)
        else:
            # print("greater")
            Suspicious_Transactions = round(random.uniform((100 + ((percentage-100)/5)*10), 300), 0)

        
        if source in ["Wire Transfers", "ACH Files"]:
            base_amount= 20
            Fraud_multiplier = math.floor(max((Transactions_value - 50),0) / 10)
            fraud_risk = round(base_amount + Fraud_multiplier * 10, 0)    
        else:
            base_amount = 50
            Fraud_multiplier = math.floor(max((Transactions_value - 50),0) / 10)
            fraud_risk = round(base_amount + Fraud_multiplier * 5, 0)
        
        

        #print(Transactions_value, Fraud_value, percentage, Suspicious Transactions, fraud_risk)

        data = {
            "state": state,
            "capital":details['capital'],
            "latitude":details['latitude'],
            "longitude":details['longitude'],
            "Number_of_Transactions": Transactions_value,
            "Fraudulent_Transactions": Fraud_value,
            "fraud_risk": fraud_risk,
            "Suspicious_Transactions": Suspicious_Transactions

        }

        return data

def simulate_Financial_data():
    start_time = datetime.now() - timedelta(days=3)
    counter = 0

    while True:
        for state, details in us_states.items():
            current_time_pas = start_time + timedelta(seconds=80 * counter)
            grid = FraudDetectionSimulator(current_time_pas)

            for src in ["Wire Transfers", "ACH Files", "Check", "ATM Transactions", "Real-Time Payments"]:
                sensor_data = grid.get_sensor_data(src, state, details)
                grid.log_event(src, sensor_data)

            counter += 1
            print(f"Counter: {counter}, Time: {current_time_pas}, state: {state}")

            # Determine if we need to sleep
            if current_time_pas >= datetime.now():
                time.sleep(80)
            else:
                time.sleep(1)
    # # grids = [FraudDetectionSimulator(grid_id=i, current_time_pas=datetime.now()) for i in range(1, 11)]
    # # grid
    
    # start_time = datetime.now() - timedelta(days=3)
    # counter = 0
    # current_time_pas = start_time + timedelta(seconds=80*counter)
    # while True:
        
    #     for state, details in us_states.items():
    #         # if datetime.now() < current_time_pas:
    #         #     current_time_pas = datetime.now()
    #         # else:
    #         current_time_pas = start_time + timedelta(seconds=80*counter)
    #         grid = FraudDetectionSimulator( current_time_pas)
    #         for src in ["Wire Transfers", "ACH Files", "Check", "ATM Transactions", "Real-Time Payments"]:
    #             sensor_data = grid.get_sensor_data(src, state, details)
    #             grid.log_event(src, sensor_data)
    #         counter = counter + 1
    #         print(counter)
    #         #self.current_time += timedelta(minutes=travel_time)

    #     sleep_time = 2
    #     #print("---------------------------------------")
    #     time.sleep(sleep_time)
    #     # if datetime.now() < current_time_pas:
    #     #     break
    #         # sleep_time = 80
    #         # time.sleep(sleep_time)

simulate_energy_data()



StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 13, Submitted, Running, Running)

Counter: 1, Time: 2025-07-08 08:04:07.936624, Country: Albania
Counter: 2, Time: 2025-07-08 08:05:27.936624, Country: Andorra
Counter: 3, Time: 2025-07-08 08:06:47.936624, Country: Austria
Counter: 4, Time: 2025-07-08 08:08:07.936624, Country: Belarus
Counter: 5, Time: 2025-07-08 08:09:27.936624, Country: Belgium
Counter: 6, Time: 2025-07-08 08:10:47.936624, Country: Bosnia and Herzegovina
Counter: 7, Time: 2025-07-08 08:12:07.936624, Country: Bulgaria
Counter: 8, Time: 2025-07-08 08:13:27.936624, Country: Croatia
Counter: 9, Time: 2025-07-08 08:14:47.936624, Country: Cyprus
Counter: 10, Time: 2025-07-08 08:16:07.936624, Country: Czech Republic
Counter: 11, Time: 2025-07-08 08:17:27.936624, Country: Denmark
Counter: 12, Time: 2025-07-08 08:18:47.936624, Country: Estonia
Counter: 13, Time: 2025-07-08 08:20:07.936624, Country: Finland
Counter: 14, Time: 2025-07-08 08:21:27.936624, Country: France
Counter: 15, Time: 2025-07-08 08:22:47.936624, Country: Germany
Counter: 16, Time: 2025-07-0

In [ ]:
from datetime import datetime, timedelta
import time

def simulate_energy_data():
    start_time = datetime.now() - timedelta(days=1)
    counter = 0

    while True:
        for country, details in all_euorpe_countries.items():
            current_time_pas = start_time + timedelta(seconds=80 * counter)
            #grid = EnergyGridSensorSimulator(current_time_pas)

            for src in ["thermal", "wind", "solar", "coal", "natural gas"]:
                print(src, country, details)
                #print(src, sensor_data)

            counter += 1
            print(f"Counter: {counter}, Time: {current_time_pas}, Country: {country}")

            # Determine if we need to sleep
            if current_time_pas >= datetime.now():
                time.sleep(80)

simulate_energy_data()

StatementMeta(, 5e19f08d-f885-4819-a3c1-39e28d5a069f, 8, Finished, Cancelled, Cancelled)

thermal Albania {'capital': 'Tirana', 'latitude': 41.3275, 'longitude': 19.8189}
wind Albania {'capital': 'Tirana', 'latitude': 41.3275, 'longitude': 19.8189}
solar Albania {'capital': 'Tirana', 'latitude': 41.3275, 'longitude': 19.8189}
coal Albania {'capital': 'Tirana', 'latitude': 41.3275, 'longitude': 19.8189}
natural gas Albania {'capital': 'Tirana', 'latitude': 41.3275, 'longitude': 19.8189}
Counter: 1, Time: 2025-07-10 07:57:42.344841, Country: Albania
thermal Andorra {'capital': 'Andorra la Vella', 'latitude': 42.5078, 'longitude': 1.5211}
wind Andorra {'capital': 'Andorra la Vella', 'latitude': 42.5078, 'longitude': 1.5211}
solar Andorra {'capital': 'Andorra la Vella', 'latitude': 42.5078, 'longitude': 1.5211}
coal Andorra {'capital': 'Andorra la Vella', 'latitude': 42.5078, 'longitude': 1.5211}
natural gas Andorra {'capital': 'Andorra la Vella', 'latitude': 42.5078, 'longitude': 1.5211}
Counter: 2, Time: 2025-07-10 07:59:02.344841, Country: Andorra
thermal Austria {'capital':

In [ ]:
import random, math, json
import random
import time
import uuid
from datetime import datetime, timedelta

class EnergyGridSensorSimulator:
    def __init__(self, current_time_pas):
        # self.grid_id = grid_id
        self.unique_id = str(uuid.uuid4())
        self.current_time = current_time_pas

    def log_event(self, activity, sensor_data):
        event = {
            "timestamp": self.current_time.strftime("%Y-%m-%d %H:%M:%S"),
            "event_id": self.unique_id,
            "source": activity,
            # "grid_id": grid_id,
            "sensor_data": sensor_data
        }
        log_event = event
        log_obj = json.dumps(log_event)
        #impressionEvent = json_object    
        sendToEventsHub(log_event, producer_events)
        #print(log_event)

    def get_sensor_data(self, source, country, details):
        sources = ["thermal", "wind", "solar", "coal", "natural gas"]
        data = {}

        # for source in sources:
        production_value = round(random.uniform(300, 500) if random.random() < 1/1000 else random.uniform(100, 300), 0)

        # Ensure consumption < production only 1 in 10 times
        if random.random() > 1/20:
            consumption_value = round(random.uniform(0.5 * production_value, production_value), 0)
        else:
            consumption_value = round(random.uniform(1.1 * production_value, 2 * production_value), 0)

        # Calculate price based on the difference
        percentage = round((consumption_value / production_value) * 100, 0)

        if percentage < 70:
            price = round(random.uniform(-10, 30), 0)
        elif percentage < 80:
            price = round(random.uniform(30, 50), 0)
        elif percentage < 90:
            price = round(random.uniform(50, 60), 0)
        elif percentage < 100:
            price = round(random.uniform(60, 100), 0)
        else:
            # print("greater")
            price = round(random.uniform((100 + ((percentage-100)/5)*10), 300), 0)

        
        base_emission = 50
        emission_multiplier = math.floor((production_value - 50) / 10)
        co2_emission = round(base_emission + emission_multiplier * 10, 0)
        

        #print(production_value, consumption_value, percentage, price, co2_emission)

        data = {
            "country": country,
            "capital":details['capital'],
            "latitude":details['latitude'],
            "longitude":details['longitude'],
            "production": production_value,
            "consumption": consumption_value,
            "co2 emission": co2_emission,
            "hourly price": price
        }

        return data

def simulate_energy_data():
    # grids = [EnergyGridSensorSimulator(grid_id=i, current_time_pas=datetime.now()) for i in range(1, 11)]
    # grid
    
    #start_time = datetime.now() - timedelta(days=3)
    counter = 0
    #current_time_pas = start_time + timedelta(seconds=80*counter)
    while True:
        
        for country, details in all_euorpe_countries.items():
            # if datetime.now() < current_time_pas:
            #     current_time_pas = datetime.now()
            # else:
            current_time_pas = datetime.now()
            grid = EnergyGridSensorSimulator( current_time_pas)
            for src in ["thermal", "wind", "solar", "coal", "natural gas"]:
                sensor_data = grid.get_sensor_data(src, country, details)
                grid.log_event(src, sensor_data)
            counter = counter + 1
            print(counter)
            #self.current_time += timedelta(minutes=travel_time)

        #sleep_time = 30
        #print("---------------------------------------")
        #time.sleep(sleep_time)
        # if datetime.now() < current_time_pas:
        #     break
            sleep_time = 80
            time.sleep(sleep_time)

simulate_energy_data()



StatementMeta(, 989badbd-8329-483e-b4ff-6bb75fc7bba8, 11, Finished, Cancelled, Cancelled)

1
2
3
4
5
6


In [ ]:
import random

# "heel": {"value": round(random.uniform(*random.choice([(-10, -5), (5, 10)])) if weather_condition else random.uniform(-5, 5), 2), "unit": "Degrees"},

def get_sensor_data():
    sources = ["thermal", "wind", "solar", "coal", "natural gas"]
    data = {}

    for source in sources:
        data[source] = {
            "production": {
                "value": round(random.uniform(100, 500) if random.random() < 1/1000 else random.uniform(50, 300), 2),
                "unit": "MW"
            },
            "consumption": {
                "value": round(random.uniform(100, 500) if random.random() < 1/1000 else random.uniform(50, 300), 2),
                "unit": "MW"
            },
            "co2 emission": {
                "value": round(random.uniform(100, 500) if random.random() < 1/1000 else random.uniform(10, 100), 2),
                "unit": "tCO2e"
            },
            "hourly price": {
                "value": round(random.uniform(100, 200) if random.random() < 1/1000 else random.uniform(30, 100), 2),
                "unit": "EUR/MWh"
            }
        }

    return data

sens = get_sensor_data()
print(sens)

StatementMeta(, cde12c39-c650-4cd1-8c9b-f00bd7f99d8f, 3, Finished, Available, Finished)

{'Thermal': {'Production': {'value': 65.04, 'unit': 'MW'}, 'Consumption': {'value': 52.44, 'unit': 'MW'}, 'CO2 Emission': {'value': 89.59, 'unit': 'tCO2e'}, 'Hourly Price': {'value': 69.47, 'unit': 'EUR/MWh'}}, 'Wind': {'Production': {'value': 118.41, 'unit': 'MW'}, 'Consumption': {'value': 179.29, 'unit': 'MW'}, 'CO2 Emission': {'value': 94.48, 'unit': 'tCO2e'}, 'Hourly Price': {'value': 31.1, 'unit': 'EUR/MWh'}}, 'Solar': {'Production': {'value': 130.79, 'unit': 'MW'}, 'Consumption': {'value': 153.34, 'unit': 'MW'}, 'CO2 Emission': {'value': 80.7, 'unit': 'tCO2e'}, 'Hourly Price': {'value': 79.3, 'unit': 'EUR/MWh'}}, 'Coal': {'Production': {'value': 214.33, 'unit': 'MW'}, 'Consumption': {'value': 188.67, 'unit': 'MW'}, 'CO2 Emission': {'value': 41.26, 'unit': 'tCO2e'}, 'Hourly Price': {'value': 33.06, 'unit': 'EUR/MWh'}}, 'Natural Gas': {'Production': {'value': 172.1, 'unit': 'MW'}, 'Consumption': {'value': 69.11, 'unit': 'MW'}, 'CO2 Emission': {'value': 72.48, 'unit': 'tCO2e'}, 'Hou